In [52]:
import csv
import cassiopeia as cass
from parameters import match_features_names


## TODO filter for remakes, use AI for role recognition

def display_player_info(player_name, region):
    player = cass.Summoner(name=player_name, region=region)
    print("{name}  is a level {level} summoner on the {region} server".format(name=player.name,
                                                                              level=player.level,
                                                                              region=player.region))
    great_with = player.champion_masteries.filter(lambda cm: cm.level >= 6)
    print("{name} is good with:".format(name=player.name))
    print([cm.champion.name for cm in great_with])


def get_match_features(match):
    participants_features = {}
    for i, participant in enumerate(match.participants):
        key = "User" + str(i)
        participants_features[key] = {feature_name: participant.stats.__getattribute__(feature_name) for feature_name in
                                      match_features_names}
    return participants_features


def get_player_history(player_name, region):
    player = cass.Summoner(name=player_name, region=region)
    return player.match_history


def current_match_to_history_index(current_match, match_history):
    for match_num, match in enumerate(match_history):
        if current_match.id == match.id:
            return match_num


def create_data_array(player_records):
    # input -> player records
    # output -> array of shape (Number of users, number_of_features, max_history_lenght)
    NotImplementedError


def get_challenger_300():
    return cass.get_challenger_league(queue=cass.data.Queue("RANKED_SOLO_5x5"), region="EUW")


class Players:
    def __init__(self):
        self.players_in_dataset = {}
        self.players_ids = []

    def add_players_to_player_ids(self, match_participants):
        for participant in match_participants:
            summoner = participant.summoner
            if summoner.puuid not in self.players_ids:
                self.players_ids.append(summoner.puuid)
                self.players_in_dataset[summoner.puuid] = {"id": len(self.players_in_dataset),
                                                           "match_history":
                                                               [match for match in summoner.match_history]}


class Matches:
    def __init__(self):
        self.matches = []
        self.ids_in_dataset = []
        self.match_id = -1

    def add_match(self, players, match):
        self.ids_in_dataset.append(match.id)
        match_arr = [match.id, int(match.start.timestamp())]
        for user_num, participant in enumerate(match.participants):
            puuid = participant.summoner.puuid
            user_id = players.players_in_dataset[puuid]['id']
            history_id = current_match_to_history_index(match, players.players_in_dataset[puuid]["match_history"])
            match_arr.append((user_id, history_id))
        self.match_id += 1  # Dla optymalizacji nie robie len() i self.matches mozna tez oprozniac
        self.matches.append(match_arr)


        return self.match_id

    def is_filter_match(self, match):
        is_in_dataset = match.id in self.ids_in_dataset
        is_remake = match.is_remake
        return is_remake or is_in_dataset


def data_csv_save(match_features, match_id):
    data = list(match_features.values())

    with open('data.csv', 'a', encoding='UTF8', newline='') as f:
        writer = csv.writer(f)
        print('data', data)
        writer.writerow([match_id] + data)
    print(f"------------> Saved {match_id}")


def data_csv_init():
    header = ['', 'User1', 'User2', 'User3', 'User4', 'User5', 'User6', 'User7', 'User8', 'User9', 'User10']

    with open('data.csv', 'w', encoding='UTF8', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(header)


def construct_matches_dataset():
    players = Players()
    matches = Matches()
    data_csv_init()
    challenger = get_challenger_300()

    for entry in challenger.entries[0:1]:
        match_history = entry.summoner.match_history[0:2]

        for i, match in enumerate(match_history):
            if matches.is_filter_match(match):
                continue

            players.add_players_to_player_ids(match.participants)
            match_id = matches.add_match(players, match)
            match_features = get_match_features(match)
            data_csv_save(match_features, matches.match_id)
    return players, matches

In [53]:
cass.set_riot_api_key("RGAPI-778b2985-d816-414a-b097-32b206234bde")
players, matches = construct_matches_dataset()

Making call: https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/quOATWbrCBX1GRkcwQEGQPk6X-I-lZHmIHcH9wgQjxQezbytiEy81nsvxvKQLz6Htg0n6e_HCuoeeA/ids?start=0&count=100
Making call: https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/t_UnMOHin7m9hjA-nsy1uGwoFBYkUa-9JQiVqg_5kzfCWz0t8QwD70v_wwdwt6Pr5bMjhD0_DsvqUQ/ids?start=0&count=100
Making call: https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/t_UnMOHin7m9hjA-nsy1uGwoFBYkUa-9JQiVqg_5kzfCWz0t8QwD70v_wwdwt6Pr5bMjhD0_DsvqUQ/ids?start=100&count=100
Making call: https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/t_UnMOHin7m9hjA-nsy1uGwoFBYkUa-9JQiVqg_5kzfCWz0t8QwD70v_wwdwt6Pr5bMjhD0_DsvqUQ/ids?start=200&count=100
Making call: https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/t_UnMOHin7m9hjA-nsy1uGwoFBYkUa-9JQiVqg_5kzfCWz0t8QwD70v_wwdwt6Pr5bMjhD0_DsvqUQ/ids?start=300&count=100
Making call: https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/t_UnMOHin7m9hjA-nsy1uGwoFBYkUa-9J

In [43]:
print(list(match_features.values()))

[{'kda': 0.5, 'largest_killing_spree': 2, 'largest_multi_kill': 1, 'killing_sprees': 1, 'longest_time_spent_living': 704, 'double_kills': 0, 'triple_kills': 0, 'quadra_kills': 0, 'penta_kills': 0, 'unreal_kills': 0, 'total_damage_dealt': 150009, 'magic_damage_dealt': 81398, 'physical_damage_dealt': 60118, 'true_damage_dealt': 8493, 'largest_critical_strike': 323, 'total_damage_dealt_to_champions': 11683, 'magic_damage_dealt_to_champions': 6108, 'physical_damage_dealt_to_champions': 5498, 'true_damage_dealt_to_champions': 76, 'total_heal': 2794, 'total_units_healed': 2, 'damage_self_mitigated': 9534, 'damage_dealt_to_objectives': 5919, 'damage_dealt_to_turrets': 4698, 'vision_score': 21, 'time_CCing_others': 6, 'total_damage_taken': 17311, 'magic_damage_taken': 5921, 'true_damage_taken': 1973, 'gold_earned': 11284, 'gold_spent': 10975, 'turret_kills': 2, 'inhibitor_kills': 0, 'total_minions_killed': 223, 'neutral_minions_killed': 10, 'total_time_cc_dealt': 168, 'vision_wards_bought': 3,

In [28]:
print(players.players_in_dataset)

{'t_UnMOHin7m9hjA-nsy1uGwoFBYkUa-9JQiVqg_5kzfCWz0t8QwD70v_wwdwt6Pr5bMjhD0_DsvqUQ': {'id': 0, 'match_history': [<cassiopeia.core.match.Match object at 0x00000168332C5D80>, <cassiopeia.core.match.Match object at 0x0000016833ED07F0>, <cassiopeia.core.match.Match object at 0x0000016833E9CA90>, <cassiopeia.core.match.Match object at 0x0000016833E9C640>, <cassiopeia.core.match.Match object at 0x0000016833E9CFD0>, <cassiopeia.core.match.Match object at 0x0000016833E9C400>, <cassiopeia.core.match.Match object at 0x0000016833E9DA50>, <cassiopeia.core.match.Match object at 0x000001681268AC50>, <cassiopeia.core.match.Match object at 0x0000016833E9D150>, <cassiopeia.core.match.Match object at 0x0000016833E9D7E0>, <cassiopeia.core.match.Match object at 0x0000016833E9D0F0>, <cassiopeia.core.match.Match object at 0x0000016833E9E380>, <cassiopeia.core.match.Match object at 0x0000016833E9E200>, <cassiopeia.core.match.Match object at 0x0000016833E9F640>, <cassiopeia.core.match.Match object at 0x00000168

In [29]:
print(players.players_ids)

['t_UnMOHin7m9hjA-nsy1uGwoFBYkUa-9JQiVqg_5kzfCWz0t8QwD70v_wwdwt6Pr5bMjhD0_DsvqUQ', 'JxIhA9AZoiplUY1ClPNU-RcRVJcUAaX-1uWKjGmcO_QfthvMDO7CrzKLLCaKCadudc1sYYdcyYIv9A', '2yP3uvolp3zvHKIdkOw7j83lX_MipuB83tFR1z5CmeZVWgX4M40XuyOuED4I3NtCQs9ldcHkquVSuQ', 'x0jM4HHuIWUNVWjTAMwr6AmPsM2RRq4kVy_SOu3pord81Xoxu055vdfF5FUhBDe2NDAplv46xigIqg', 'hh4CkAc2a6FH82Wsjlth26v6L6C7zbAGuB7C-eIM0RDom0UdSLm1ltAzrcGJoIF-6aTPMmDQhaejTw', 'aFzl2B90qbakRErJC6diijm2V5HgWEjRD5Gjq0hk-1ASqMEdiFSvlW_bkmc2VwGPMU_XqZLh5DEFfA', 'MvdUwlQ-P9e_m3FiA2ziIZWkbOBLVrI99TDQJgCCyEMgi_YTFYeVY-_6LFPeOBjJsK7MHY_lORE9Lw', 'ZyagjgNOOTuCCQaNoYMd61cskMq-jJ75yGTV5cXFFxo4DzGl87JYA4vFt_QF5_13Efxw1HnwWLuVxw', 'HRovrQKXuXXXj6-z_2o93hHR7FTufnA7GsQpnn4jF5WFlJTLl40mUmIjlAig9wc05xtopByXbbxRrA', 'quOATWbrCBX1GRkcwQEGQPk6X-I-lZHmIHcH9wgQjxQezbytiEy81nsvxvKQLz6Htg0n6e_HCuoeeA']


In [30]:
print(matches.matches)

[[6191487361, 1671194017, (0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (5, 0), (6, 0), (7, 0), (8, 0), (9, 0)]]


In [31]:
print(matches.ids_in_dataset)

[6191487361]


In [36]:
def save_matches(data):
    header = ['', 'match_id', 'time', 'User1', 'User2', 'User3', 'User4', 'User5', 'User6', 'User7', 'User8', 'User9',
              'User10']

    with open('train.csv', 'w', encoding='UTF8', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(header)
        for i, x in enumerate(data):
            print(x)
            writer.writerow([i] + x)


save_matches(matches.matches)

[6191487361, 1671194017, (0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (5, 0), (6, 0), (7, 0), (8, 0), (9, 0)]
